
## Setup 

First, we need to install .NET:

- On Windows, [install the .NET framework](https://msdn.microsoft.com/en-us/library/5a4x27ek.aspx).
- On Mac or Linux, install [mono](http://www.mono-project.com) (on Mac, if you have _Homebrew_, just do `brew install mono`).

Next, install [Python for .NET](http://pythonnet.github.io): `pip install pythonnet`. If you are having issues, try `pip install pycparser` on Linux/Mac, and try to get the latest code from [github](https://github.com/pythonnet/pythonnet) and install with `python setup.py install`.

Now, check that it works - this import should succeed:

In [1]:
import clr

## Simple examples

In [2]:
from System import String

In [3]:
s = String('hi')
s.get_Length()

2

In [4]:
from System import Environment

In [5]:
Environment.MachineName

'SilverBullet.home'

In [6]:
clr.AddReference('System.Collections');
from System.Collections import Hashtable

In [14]:
hashtable = Hashtable()
hashtable.Add('Python', 6)
hashtable.Add('.NET', 4)

hashtable.ContainsKey('Python'), hashtable['Python']

(True, 6)

## Example: XML writer

Write an XML writer using the .NET XMLTextWriter class ([source](https://www.thecodingforums.com/threads/example-code-python-pythonnet-ironpython-boo.336422/)).

In [ ]:
import clr
import System
from System import Console as CLR_Console
import System.Xml as CLR_Xml

In [17]:
true = 1
false = 0
filename = "tmp.xml"
writer = CLR_Xml.XmlTextWriter(filename, None);
#~ Use indenting for readability.
writer.Formatting = CLR_Xml.Formatting.Indented;

writer.WriteComment("XML in Boo Language");

#~ Write an element (this one is the root).
writer.WriteStartElement("EducationalCentres");

#~ Write the namespace declaration.
writer.WriteAttributeString("xmlns", "schname", None,
"urn:schoolnames");

writer.WriteStartElement("School");

#~ Lookup the prefix and then write the ISBN attribute.
prefix = writer.LookupPrefix("urn:schoolnames");
writer.WriteStartAttribute(prefix, "NAME", "urn:schoolnames");
writer.WriteString("ABCD International School");
writer.WriteEndAttribute(); 

#~ Write the title.
writer.WriteStartElement("city");
writer.WriteString("Madurai");
writer.WriteEndElement();

#~ Write the price.
writer.WriteElementString("Students", "500");

#~ Write the style element.
writer.WriteStartElement(prefix, "Syllabus", "urn:schoolnames");
writer.WriteString("IGCSE");
writer.WriteEndElement();

#~ Write the end tag for the book element.
writer.WriteEndElement();

#~ Write the close tag for the root element.
writer.WriteEndElement();

#~ Write the XML to file and close the writer.
writer.Flush();
writer.Close();

#~ Read the file back in and parse to ensure well formed XML.
doc = CLR_Xml.XmlDocument();
#~ Preserve white space for readability.
doc.PreserveWhitespace = true;
#~ Load the file
doc.Load(filename);

#~ Write the XML content to the console.
CLR_Console.Write(doc.InnerXml);

In [19]:
%cat test1.xml

<!--XML in Boo Language-->
<EducationalCentres xmlns:schname="urn:schoolnames">
  <School schname:NAME="ABCD International School">
    <city>Madurai</city>
    <Students>500</Students>
    <schname:Syllabus>IGCSE</schname:Syllabus>
  </School>
</EducationalCentres>

## Example: load C# library

Start by writing a C# library. We'll write a simple Fibonacci number calculator:

In [57]:
%%file FibonacciExample.cs
using System;

namespace Com.Yoavram.Examples
{
   public class FibonacciExample
   {
      public static int Fibonacci(int n)
      {
          int a = 1;
          int b = 1;
          int i = 2;
          while (i < n) {
              int tmp = b;
              b = a + b;
              a = tmp;
              i++;
          }
          return b;
      }
      public static void Main(string[] args)
      {
         Console.WriteLine("Fibonacci 10:");
         Console.WriteLine(Fibonacci(10));
      }
   } 
}

Overwriting FibonacciExample.cs


Now to compile it and run it on MacOS (running is calling the `Main` function):

In [6]:
!mcs FibonacciExample.cs
!mono FibonacciExample.exe

Fibonacci 10:
55


Now we load the libary to Python and import the namespace:

In [13]:
assembly = clr.AddReference('FibonacciExample')
from Com.Yoavram.Examples import FibonacciExample

so that we can use the functions from the namespace:

In [18]:
FibonacciExample.Fibonacci(10)

55

Let's compete the .NET Fibonacci function with a Python one:

In [14]:
def fibonacci(n):
    a = 1
    b = 1
    i = 2
    while i < n:
        a, b = b, a + b
        i+= 1
    return b

In [15]:
fibonacci(10)

55

In [17]:
%timeit fibonacci(100)
%timeit FibonacciExample.Fibonacci(100)

100000 loops, best of 3: 8.43 µs per loop
The slowest run took 20.84 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.76 µs per loop


Alright, the .NET implementation is faster!